In [1]:
import pandas as pd
import _pickle as pickle

In [2]:
table = pd.read_csv('./icd9toicd10cmgem.csv')
gem_icd9toicd10 = {}
gem_icd10toicd9 = {}

for row in table.iterrows():
    gem_icd9toicd10[row[1].icd9cm] = row[1].icd10cm
    gem_icd10toicd9[row[1].icd10cm] = row[1].icd9cm

In [3]:
df = pd.read_excel('epicicd9toicd10.xlsx')

icd10toicd9 = {}
icd9_list = []
icd10_list = []

for index, row in df.iterrows():
    icd9 = row.ICD9
    icd10 = row.ICD10
    number = row.Count
    icd10_list.extend(icd10.split(', '))
    icd9_list.extend(icd9.split(', '))

    if ',' in icd10:
        for c in icd10.split(', '):
            if c in icd10toicd9:
                icd10toicd9[c].append((icd9,number))
            else:
                icd10toicd9[c] = [(icd9,number)]
    else:
        if ',' in icd9:
            for c in icd9.split(', '):
                if icd10 in icd10toicd9:
                    icd10toicd9[icd10].append((c,number))
                else:
                    icd10toicd9[icd10] = [(c,number)]
        else:
            if icd10 in icd10toicd9:
                icd10toicd9[icd10].append((icd9,number))
            else:
                icd10toicd9[icd10] = [(icd9,number)]  
                
icd10_list = list(set(icd10_list))
icd9_list = list(set(icd9_list))

len(icd10_list), len(icd9_list)

nch_icd10toicd9 = {}
for key, value in icd10toicd9.items():
    nch_icd10toicd9[key.replace('.','')] = sorted(value, key=lambda x: x[1], reverse=True)[0][0].replace('.','')

In [4]:
pickle.dump(nch_icd10toicd9, open('nch_icd10toicd9','wb'))
pickle.dump(gem_icd10toicd9, open('gem_icd10toicd9','wb'))

In [ ]:
# search gem mapping first
def search_similar(code):
    length = len(code)
    while length >=3:
        for key, value in gem_icd10toicd9.items():
            if key.startswith(code[:length]):
                return key
        length -=1
    length = len(code)
    while length >=3:
        for key, value in nch_icd10toicd9.items():
            if key.startswith(code[:length]):
                return key
        length -=1
    return 'NotExist'